# Extract, Transform and Load
<hr>

## Dependencies

In [1]:
# Dependencies and libraries

import pandas as pd
import datetime as dt
from sqlalchemy import create_engine
import requests
import pymongo

## Movies Extract

In [2]:
# Creates the dataframe
movies_file="MoviesOnStreamingPlatforms.csv"
movies_df=pd.read_csv(movies_file)

In [3]:
movies_df.columns

Index(['Unnamed: 0', 'ID', 'Title', 'Year', 'Age', 'IMDb', 'Rotten Tomatoes',
       'Netflix', 'Hulu', 'Prime Video', 'Disney+', 'Type', 'Directors',
       'Genres', 'Country', 'Language', 'Runtime'],
      dtype='object')

In [4]:
# Selecting and renaming relevant columns
new_movies_df=movies_df[['Title', 'Year','Directors', 'Age','Genres', 'Language','Hulu', 'Prime Video', 'Disney+']].copy()
new_movies_df.rename(columns={'Title':'title','Year':'year','Genres':'genre','Language':'languages','Directors':'director', 'Age':'age', 'Hulu':'hulu',
                              'Prime Video':'prime_video', 'Disney+':'disney_plus'},inplace=True)
new_movies_df.head()


,title,year,director,age,genre,languages,hulu,prime_video,disney_plus
0,Inception,2010,Christopher Nolan,13+,"Action,Adventure,Sci-Fi,Thriller","English,Japanese,French",0,0,0
1,The Matrix,1999,"Lana Wachowski,Lilly Wachowski",18+,"Action,Sci-Fi",English,0,0,0
2,Avengers: Infinity War,2018,"Anthony Russo,Joe Russo",13+,"Action,Adventure,Sci-Fi",English,0,0,0
3,Back to the Future,1985,Robert Zemeckis,7+,"Adventure,Comedy,Sci-Fi",English,0,0,0
4,"The Good, the Bad and the Ugly",1966,Sergio Leone,18+,Western,Italian,0,1,0


Create our key with movie title and movie year

In [5]:
new_movies_df['key']=new_movies_df['title'].astype('string')+"-"+new_movies_df['year'].astype(str)


## Netflix Movies Extract

In [6]:
# Creating dataframe, this will be the main one to work on

netflix_file="netflix-dataset.csv"
netflix_df=pd.read_csv(netflix_file)

## Netflix Transformation

In [7]:
netflix_df.columns

Index(['Title', 'Genre', 'Tags', 'Languages', 'Series or Movie',
       'Hidden Gem Score', 'Country Availability', 'Runtime', 'Director',
       'Writer', 'Actors', 'View Rating', 'IMDb Score',
       'Rotten Tomatoes Score', 'Metacritic Score', 'Awards Received',
       'Awards Nominated For', 'Boxoffice', 'Release Date',
       'Netflix Release Date', 'Production House', 'Netflix Link', 'IMDb Link',
       'Summary', 'IMDb Votes', 'Image', 'Poster', 'TMDb Trailer',
       'Trailer Site'],
      dtype='object')

### We search only for Movies in Netflix Dataset

In [8]:
# Slicing the dataframe to contain only movies

new_netflix_df = netflix_df[netflix_df['Series or Movie']=='Movie'].copy()

In [9]:
# Selecting and renaming relevant columns

new_netflix_df=new_netflix_df[['Title', 'Genre', 'Languages', 'Director','Actors', 'IMDb Score', 
                'Release Date', 'IMDb Link', 'Poster','Summary']]

new_netflix_df.rename(columns = {'Title':'title', 'Genre':'genre', 'Languages':'languages', 
                               'Director':'director','Actors':'actors', 'IMDb Score':'imdb_score', 
                               'Release Date':'release_date', 'IMDb Link':'imdb_link',
                               'Summary':'summary', 'Poster':'poster'},inplace=True)



Define all movies in Netflix dataset with 1 value in Netflix column

In [10]:
new_netflix_df['netflix'] = 1

Format date column to date format and extract the year

In [11]:
# Casting release date to datetime datatype and extracting the year only

new_netflix_df['release_date'] = pd.to_datetime(new_netflix_df['release_date'], format='%d %b %Y')
new_netflix_df.dropna(subset=['release_date'],inplace=True)
new_netflix_df['year']=new_netflix_df.loc[:,'release_date'].dt.year.astype(int)

In [12]:
new_netflix_df=new_netflix_df[['title','year', 'netflix', 'genre', 'languages', 'director', 'actors',
       'imdb_score', 'release_date', 'imdb_link', 'poster', 'summary']]

### Drop movies with the same title

Delete all movies with the same title as we will use movie title and movie year as our join key

In [13]:
new_netflix_df = new_netflix_df.drop_duplicates(subset='title',keep='first')

print(f"There are {new_netflix_df['title'].duplicated().sum()} duplicate titles in the Netflix dataset")

There are 0 duplicate titles in the Netflix dataset


Create our key with movie title and movie year

In [15]:
new_netflix_df['key']=new_netflix_df['title']+"-"+new_netflix_df['year'].astype(str)

In [16]:
new_netflix_df.head(3)

,title,year,netflix,genre,languages,director,actors,imdb_score,release_date,imdb_link,poster,summary,key
1,HOW TO BUILD A GIRL,2020,1,Comedy,English,Coky Giedroyc,"Paddy Considine, Cleo, Beanie Feldstein, Dónal...",5.8,2020-05-08,https://www.imdb.com/title/tt4193072,https://m.media-amazon.com/images/M/MV5BZGUyN2...,"When nerdy Johanna moves to London, things get...",HOW TO BUILD A GIRL-2020
2,Centigrade,2020,1,"Drama, Thriller",English,Brendan Walsh,"Genesis Rodriguez, Vincent Piazza",4.3,2020-08-28,https://www.imdb.com/title/tt8945942,https://m.media-amazon.com/images/M/MV5BODM2MD...,"Trapped in a frozen car during a blizzard, a p...",Centigrade-2020
4,Moxie,2011,1,"Animation, Short, Drama",English,Stephen Irwin,Ragga Gudrun,6.3,2011-09-22,https://www.imdb.com/title/tt2023611,https://m.media-amazon.com/images/M/MV5BODYyNW...,Inspired by her moms rebellious past and a con...,Moxie-2011


## Merge both data sets

In [17]:
new_bd=pd.merge(new_netflix_df,new_movies_df, how='outer', on='key')

## Some cleanning and format fix

In [18]:
# Creating a temporal dataframe for testing purposes

new_bd['title_x'] = new_bd['title_x'].fillna(0)

temporal2=new_bd.loc[new_bd['title_x']!=0,:].copy()
temporal=new_bd.loc[new_bd['title_x']==0,:].copy()

temporal['title_x']=temporal['title_y']
temporal['year_x']=temporal['year_y']
temporal['director_x']=temporal['director_y']
temporal['genre_x']=temporal['genre_y']
temporal['languages_x']=temporal['languages_y']
temporal['poster']='https://www.google.com/imgres?imgurl=https://www.athousakis.gr/images/usrImage/29-04-2021-10-25-608a5f517a0b8.jpg&imgrefurl=https://www.athousakis.gr/en/kapaki/cylinder-head-md305479-g6nfr-for-mitsubishi-6.html&tbnid=Xg9vytX9NtnT4M&vet=1&docid=2nvl95VUznhloM&w=275&h=275&itg=1&source=sh/x/im'
temporal['imdb_link']='No info available'
temporal['netflix']=0
temporal.head()

,title_x,year_x,netflix,genre_x,languages_x,director_x,actors,imdb_score,release_date,imdb_link,...,key,title_y,year_y,director_y,age,genre_y,languages_y,hulu,prime_video,disney_plus
10141,Avengers: Infinity War,2018.0,0,"Action,Adventure,Sci-Fi",English,"Anthony Russo,Joe Russo",NaN,NaN,NaT,No info available,...,Avengers: Infinity War-2018,Avengers: Infinity War,2018.0,"Anthony Russo,Joe Russo",13+,"Action,Adventure,Sci-Fi",English,0.0,0.0,0.0
10142,"The Good, the Bad and the Ugly",1966.0,0,Western,Italian,Sergio Leone,NaN,NaN,NaT,No info available,...,"The Good, the Bad and the Ugly-1966","The Good, the Bad and the Ugly",1966.0,Sergio Leone,18+,Western,Italian,0.0,1.0,0.0
10143,The Pianist,2002.0,0,"Biography,Drama,Music,War","English,German,Russian",Roman Polanski,NaN,NaN,NaT,No info available,...,The Pianist-2002,The Pianist,2002.0,Roman Polanski,18+,"Biography,Drama,Music,War","English,German,Russian",0.0,1.0,0.0
10144,Raiders of the Lost Ark,1981.0,0,"Action,Adventure","English,German,Hebrew,Spanish,Arabic,Nepali",Steven Spielberg,NaN,NaN,NaT,No info available,...,Raiders of the Lost Ark-1981,Raiders of the Lost Ark,1981.0,Steven Spielberg,7+,"Action,Adventure","English,German,Hebrew,Spanish,Arabic,Nepali",0.0,0.0,0.0
10145,Taxi Driver,1976.0,0,"Crime,Drama","English,Spanish",Martin Scorsese,NaN,NaN,NaT,No info available,...,Taxi Driver-1976,Taxi Driver,1976.0,Martin Scorsese,18+,"Crime,Drama","English,Spanish",0.0,0.0,0.0


In [19]:
# Merging the 2 temporal dataframes
good_df=pd.concat([temporal2,temporal])

# Dropiing duplicated movies
good_df= good_df.drop_duplicates(subset='key',keep='first')

good_df=good_df[['title_x', 'year_x', 'genre_x', 'languages_x', 'director_x','actors','age', 
       'netflix', 'hulu', 'prime_video', 'disney_plus','imdb_score', 'imdb_link', 'poster','summary']]
good_df['hulu'] = good_df['hulu'].fillna(0).astype(int)
good_df['prime_video'] = good_df['prime_video'].fillna(0).astype(int)
good_df['disney_plus'] = good_df['disney_plus'].fillna(0).astype(int)
good_df['year_x']=good_df['year_x'].astype(int)
good_df['netflix']=good_df['netflix'].astype(int)

# Renaming columns for clarity, and getting ready to export to Mongo
good_df=good_df.rename(columns={'title_x':'title','year_x':'year','genre_x':'genre','languages_x':'languages','director_x':'director'})
good_df.head(2)

,title,year,genre,languages,director,actors,age,netflix,hulu,prime_video,disney_plus,imdb_score,imdb_link,poster,summary
0,HOW TO BUILD A GIRL,2020,Comedy,English,Coky Giedroyc,"Paddy Considine, Cleo, Beanie Feldstein, Dónal...",NaN,1,0,0,0,5.8,https://www.imdb.com/title/tt4193072,https://m.media-amazon.com/images/M/MV5BZGUyN2...,"When nerdy Johanna moves to London, things get..."
1,Centigrade,2020,"Drama, Thriller",English,Brendan Walsh,"Genesis Rodriguez, Vincent Piazza",NaN,1,0,0,0,4.3,https://www.imdb.com/title/tt8945942,https://m.media-amazon.com/images/M/MV5BODM2MD...,"Trapped in a frozen car during a blizzard, a p..."


### Create Mongodb connection

In [20]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# Define database and collection
db = client.ETL_project
collection = db.Movies

### Load DataFrames into database

In [21]:
# Making sure database is empty
collection.drop()

# Creating the dictionary for Mongo
# We want title as object, hence the for loop

# Initializing variables and loop parameters
rows=len(good_df)
movies_dict={}
x=0 # This is just a counter to make sure loop parameters are correct

for row in range(0,rows):
    movies_dict={'title':good_df.iloc[row,:][0],
                'year':(good_df.iloc[row,:][1]).astype(str),
                'genre':good_df.iloc[row,:][2],
                 'languages':good_df.iloc[row,:][3],
                 'director':good_df.iloc[row,:][4],
                 'actors':good_df.iloc[row,:][5],
                 'age':good_df.iloc[row,:][6],
                 'netflix':good_df.iloc[row,:][7].astype(str),
                 'hulu':good_df.iloc[row,:][8].astype(str),
                 'prime_video':good_df.iloc[row,:][9].astype(str),
                 'disney_plus':good_df.iloc[row,:][10].astype(str),
                 'imdb_score':good_df.iloc[row,:][11],
                 'imdb_link':good_df.iloc[row,:][12],
                 'poster':good_df.iloc[row,:][13],
                 'summary':good_df.iloc[row,:][14]}
    x += 1
    collection.insert_one(movies_dict)

print(x)

24681
